In [1]:
import sqlite3

conn = sqlite3.connect("musica.sqlite") # hace un coceecion con el archio si este no esiste  crea uno con ese nombre
cur=conn.cursor()  #funnciona con un manejador de archivos pára enviar y recibir informacion 

cur.execute("DROP TABLE IF EXISTS Canciones") #elimina la tabla si es que ya existe 
cur.execute("CREATE TABLE Canciones(titulo TEXT, reproducciones INTEGER)")

conn.close()

In [10]:
import sqlite3
conn = sqlite3.connect("musica.sqlite")
cur=conn.cursor()

cur.execute("INSERT INTO Canciones(titulo,reproducciones)VALUES(?,?)",("Thunderstruck",20))
cur.execute("INSERT INTO Canciones(titulo,reproducciones)Values(?,?)",("My way",15))
conn.commit()

print("canciones:")
cur.execute("SELECT titulo, reproducciones FROM Canciones")
for fila in cur: #para cada fila en caciones imprime esa tupla
    print(fila)

cur.execute("DELETE FROM Canciones WHERE reproducciones <100")
conn.commit()
cur.close

canciones:
('Thunderstruck', 20)
('My way', 15)


<function Cursor.close()>

En esta sección, crearemos un programa araña sencillo que se moverá a través de
cuentas de Twitter y construirá una base de datos de ellas.

In [12]:
import urllib.request, urllib.parse, urllib.error
import oauth
import hidden

# https://apps.twitter.com/
# Create App and get the four strings, put them in hidden.py

def aumentar(url, parameters): 
    secrets = hidden.oauth()
    consumer = oauth.OAuthConsumer(secrets['consumer_key'],
                                   secrets['consumer_secret'])
    token = oauth.OAuthToken(secrets['token_key'], secrets['token_secret'])

    oauth_request = oauth.OAuthRequest.from_consumer_and_token(consumer,
                    token=token, http_method='GET', http_url=url,
                    parameters=parameters)
    oauth_request.sign_request(oauth.OAuthSignatureMethod_HMAC_SHA1(),
                               consumer, token)
    return oauth_request.to_url()


def test_me():
    print('* Calling Twitter...')
    url = aumentar('https://api.twitter.com/1.1/statuses/user_timeline.json',
                  {'screen_name': 'drchuck', 'count': '2'})
    print(url)
    connection = urllib.request.urlopen(url)
    data = connection.read()
    print(data)
    headers = dict(connection.getheaders())
    print(headers)

In [17]:
from urllib.request import urlopen
import urllib.error
import twurl
import json
import sqlite3
import ssl

TWITTER_URL = 'https://api.twitter.com/1.1/friends/list.json'

conn = sqlite3.connect('arana.sqlite')
cur = conn.cursor()

cur.execute('''
            CREATE TABLE IF NOT EXISTS Twitter
            (nombre TEXT, recuperado INTEGER, amigos INTEGER)''')

# Ignorar errores de certificado SSL
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

while True:
    cuenta = input('Ingresa una cuenta de Twitter, o salir: ')
    if (cuenta == 'salir'): break
    if (len(cuenta) < 1):
        cur.execute('''SELECT nombre FROM Twitter 
            WHERE recuperado = 0 LIMIT 1''')
        try:
            cuenta = cur.fetchone()[0]
        except:
            print('No se han encontrado cuentas de Twitter por recuperar')
            continue

    url = twurl.augment(TWITTER_URL, {'screen_name': cuenta, 'count': '5'})
    print('Recuperando', url)
    conexion = urlopen(url, context=ctx)
    datos = conexion.read().decode()
    cabeceras = dict(conexion.getheaders())

    print('Restante', cabeceras['x-rate-limit-remaining'])
    js = json.loads(datos)
    # DepuraciÃ³n
    # print json.dumps(js, indent=4)

    cur.execute('''UPDATE Twitter 
            SET recuperado=1 WHERE nombre = ?''', (cuenta, ))

    contnuevas = 0
    contantiguas = 0
    for u in js['users']:
        amigo = u['screen_name']
        print(amigo)
        cur.execute('SELECT amigos FROM Twitter WHERE nombre = ? LIMIT 1',
                    (amigo, ))
        try:
            contador = cur.fetchone()[0]
            cur.execute('UPDATE Twitter SET amigos = ? WHERE nombre = ?',
                        (contador+1, amigo))
            contantiguas = contantiguas + 1
        except:
            cur.execute('''INSERT INTO Twitter (nombre, recuperado, amigos)
                        VALUES (?, 0, 1)''', (amigo, ))
            contnuevas = contnuevas + 1
    print('Cuentas nuevas=', contnuevas, ' ya visitadas=', contantiguas)
    conn.commit()

cur.close()

AttributeError: module 'oauth' has no attribute 'OAuthConsumer'

In [15]:
!pip install twurl


ERROR: Could not find a version that satisfies the requirement twurl (from versions: none)
ERROR: No matching distribution found for twurl
